In [22]:
import os
os.chdir(r'C:\Users\Navid\Documents\ds_salary_proj')

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [24]:
df = pd.read_csv('eda_data_modified')

In [25]:
df['seniority'].value_counts()

0    1674
1     449
Name: seniority, dtype: int64

In [26]:
# df = df.loc[df.Industry != '-1']
df = df.loc[df['Type of ownership'] != 'Unknown']

In [27]:
df = df.loc[(df.Size != '-1') & (df.Size != 'Unknown')]

In [28]:
df.columns

Index(['Unnamed: 0', 'index', 'Job Title', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Size', 'Founded', 'Type of ownership',
       'Industry', 'Sector', 'Revenue', 'Competitors', 'per_hour',
       'avg_salary', 'FAANG', 'state', 'desc_length', 'seniority', 'title',
       'python', 'ML', 'big_data', 'deep_learning', 'cloud_computing',
       'graduate', 'undergrad', 'job_requirement', 'exp_year'],
      dtype='object')

In [29]:
df_final = df[['avg_salary' ,'Size','Type of ownership', 'Rating', 
               'Sector', 'Revenue','per_hour', 'seniority', 
               'state', 'cloud_computing', 'title', 'big_data', 
       'deep_learning', 'graduate', 'undergrad','exp_year']]

In [30]:
df = df[df.Rating != -1]

In [31]:
#removing skewness
# from scipy import stats
# df['avg_salary'] ,fitted_lambda1 = stats.boxcox(df['avg_salary'])

In [32]:
df_dummy = pd.get_dummies(df_final)

In [33]:
# df_dummy.drop('Industry_other_industries',axis=1,inplace=True)

In [34]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# df[['avg_salary', 'Rating', 'exp_year']] = scaler.fit_transform(df[['avg_salary', 'Rating', 'exp_year']])

In [35]:
# train test split
from sklearn.model_selection import train_test_split
x = df_dummy.drop('avg_salary' , axis = 1)
y = df_dummy['avg_salary'].values
X_train , X_test , y_train , y_test = train_test_split(x , y , test_size = 0.2 , random_state = 1) 

In [49]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
lr = SVR(C=10).fit(X_train,y_train)
cross_val_score(lr,X_train,y_train,scoring='neg_mean_absolute_error',cv=5).mean()

-15.251352163424553

In [37]:
import statsmodels.api as sm
X_sm = sm.add_constant(x)
model = sm.OLS(y,X_sm)
model.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.668
Model:                            OLS   Adj. R-squared:                  0.649
Method:                 Least Squares   F-statistic:                     35.36
Date:                Sat, 07 Nov 2020   Prob (F-statistic):               0.00
Time:                        12:39:47   Log-Likelihood:                -9339.0
No. Observations:                2122   AIC:                         1.891e+04
Df Residuals:                    2007   BIC:                         1.956e+04
Df Model:                         114                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
const                                               44.6764      2.965     15.067      0.000      38.861      50.492
Rating                                               3.9775      0.865      4.599      0.000       2.281       5.674
per_hour                                           -13.1116     10.425     -1.258      0.209     -33.557       7.334
seniority                                           22.3474      1.177     18.994      0.000      20.040      24.655
cloud_computing                                      2.1055      1.068      1.972      0.049       0.012       4.199
big_data                                             2.9459      1.275      2.311      0.021       0.446       5.446
deep_learning                                       -2.2304      1.802     -1.237      0.216      -5.765       1.304
graduate                                             2.3370      1.019      2.292      0.022       0.338       4.336
undergrad                                           -5.2448      0.990     -5.299      0.000      -7.186      -3.304
exp_year                                             0.9446      0.192      4.920      0.000       0.568       1.321
Size_1 to 50 Employees                               2.6682      2.598      1.027      0.305      -2.427       7.763
Size_10000+ Employees                                4.3956      1.688      2.605      0.009       1.086       7.705
Size_1001 to 5000 Employees                          6.4425      1.191      5.409      0.000       4.107       8.778
Size_201 to 500 Employees                            7.0451      1.335      5.277      0.000       4.427       9.663
Size_5001 to 10000 Employees                         8.5393      1.778      4.803      0.000       5.052      12.026
Size_501 to 1000 Employees                           8.2059      1.388      5.911      0.000       5.483      10.929
Size_51 to 200 Employees                             7.3797      1.504      4.908      0.000       4.431      10.328
Type of ownership_College / University              -1.8042      5.914     -0.305      0.760     -13.403       9.795
Type of ownership_Company - Private                  5.4782      2.794      1.960      0.050      -0.002      10.959
Type of ownership_Company - Public                   7.0012      2.897      2.416      0.016       1.319      12.684
Type of ownership_Contract                           2.2991     11.341      0.203      0.839     -19.942      24.540
Type of ownership_Government                        -0.9075      4.589     -0.198      0.843      -9.908       8.093
Type of ownership_Hospital                          -3.4302      5.530     -0.620      0.535     -14.274       7.414
Type of ownership_Nonprofit Organization 

## Lets start with random grid search to narrow down the ranges

In [39]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, 
                               cv = 3, verbose=2, random_state=42, n_jobs = -1, scoring = 'neg_mean_absolute_error')
# Fit the random search model
rf_random.fit(X_train, y_train)




Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   50.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  7.9min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [40]:
print(rf_random.best_score_,rf_random.best_estimator_)

-11.358091206043818 RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=400, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)


In [47]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'max_features': 'sqrt',
    'n_estimators': range(350,580,10)
}
# Create a based model
rf = RandomForestRegressor()
grid = GridSearchCV(RandomForestRegressor(n_jobs=-1 ,random_state = 42, bootstrap = False, max_features =  'sqrt'), 
                    param_grid = grid_param, scoring = 'neg_mean_absolute_error',  cv =3).fit(X_train , y_train)

In [48]:
print(grid.best_score_,grid.best_estimator_)

-11.337811947202534 RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=480, n_jobs=-1, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)


In [51]:
df_dummy.corr()['avg_salary'].sort_values(ascending =False)[0:40]

avg_salary                                   1.000000
title_data scientist                         0.436119
seniority                                    0.352128
state_CA                                     0.320150
title_machine learning engineer              0.292092
big_data                                     0.278507
deep_learning                                0.272835
cloud_computing                              0.245095
Sector_Information Technology                0.181914
title_data engineer                          0.172866
exp_year                                     0.170515
Sector_Biotech & Pharmaceuticals             0.157423
graduate                                     0.156487
Revenue_$10+ billion (USD)                   0.143223
Rating                                       0.138787
Type of ownership_Company - Public           0.127939
state_MA                                     0.100431
title_data architect                         0.096979
Revenue_Unknown / Non-Applic

In [50]:
grid.score(X_test, y_test)

-10.580224809368199